In [1]:
import pandas as pd
import numpy as np 
from sklearn.cluster import KMeans
import pulp

In [2]:
import zipfile
import os

# Define the path to the uploaded zip file
zip_path = r"C:\Users\gudal\Downloads\optimal ambualance  Datasets.zip"
extract_path = "extracted_data"

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# List extracted files
extracted_files = []
for root, dirs, files in os.walk(extract_path):
    for file in files:
        extracted_files.append(os.path.join(root, file))

extracted_files


['extracted_data\\accidents.csv',
 'extracted_data\\accidents2.csv',
 'extracted_data\\class_data.csv',
 'extracted_data\\combined_dataset.csv',
 'extracted_data\\final.csv']

In [3]:
import pandas as pd
df = pd.read_csv(r"C:\Users\gudal\Downloads\ambulace optimization\extracted_data\final.csv")
accident_df = df[df['class'] == 1].copy()
accident_coords = accident_df[['latitude', 'longitude']].values

In [8]:
num_stations = 100
kmeans = KMeans(n_clusters=num_stations, random_state=42)
kmeans.fit(accident_coords)
station_coords = kmeans.cluster_centers_

In [9]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

# Approx travel time in minutes
avg_speed_kmph = 40
num_accidents = len(accident_df)
travel_time = np.zeros((num_stations, num_accidents))

for i, (s_lat, s_lon) in enumerate(station_coords):
    for j, (a_lat, a_lon) in enumerate(accident_coords):
        dist_km = haversine(s_lat, s_lon, a_lat, a_lon)
        travel_time[i][j] = (dist_km / avg_speed_kmph) * 60

In [10]:
model = pulp.LpProblem("Ambulance_Placement", pulp.LpMinimize)

# Decision Variables
x = pulp.LpVariable.dicts("PlaceAmbulance", range(num_stations), cat='Binary')  # whether ambulance is placed
y = pulp.LpVariable.dicts("Assign", [(i, j) for i in range(num_stations) for j in range(num_accidents)], cat='Binary')

# Objective: Minimize total travel time
model += pulp.lpSum([travel_time[i][j] * y[(i, j)] for i in range(num_stations) for j in range(num_accidents)])

# Constraint: Total number of ambulances
max_ambulances = 100
model += pulp.lpSum([x[i] for i in range(num_stations)]) == max_ambulances, "MaxAmbulances"

# Constraint: Each accident is served by exactly one station
for j in range(num_accidents):
    model += pulp.lpSum([y[(i, j)] for i in range(num_stations)]) == 1, f"ServeAccident_{j}"


# Constraint: Assignments only if ambulance is placed
for i in range(num_stations):
    for j in range(num_accidents):
        model += y[(i, j)] <= x[i], f"Link_{i}_{j}"

# Constraint: Each station can serve only up to its capacity
station_capacity = 100  # Can be customized per station
for i in range(num_stations):
    model += pulp.lpSum([y[(i, j)] for j in range(num_accidents)]) <= station_capacity, f"Capacity_{i}"


In [11]:
model.solve()
print("Status:", pulp.LpStatus[model.status])

Status: Optimal


In [12]:
print("\n Ambulances Placed At:")
for i in range(num_stations):
    if x[i].varValue == 1:
        lat, lon = station_coords[i]
        print(f" - Station {i} at (lat: {lat:.5f}, lon: {lon:.5f})")


 Ambulances Placed At:
 - Station 0 at (lat: -1.26276, lon: 36.84029)
 - Station 1 at (lat: -2.07988, lon: 37.47302)
 - Station 2 at (lat: -1.17987, lon: 36.93493)
 - Station 3 at (lat: -1.35571, lon: 36.90750)
 - Station 4 at (lat: -0.98387, lon: 36.63075)
 - Station 5 at (lat: -1.02935, lon: 37.06862)
 - Station 6 at (lat: -1.26153, lon: 36.71980)
 - Station 7 at (lat: -1.31867, lon: 36.80603)
 - Station 8 at (lat: -1.52452, lon: 37.12514)
 - Station 9 at (lat: -2.10000, lon: 36.75000)
 - Station 10 at (lat: -2.66757, lon: 37.87949)
 - Station 11 at (lat: -0.90640, lon: 37.70654)
 - Station 12 at (lat: -0.59498, lon: 36.98525)
 - Station 13 at (lat: -1.27468, lon: 37.29636)
 - Station 14 at (lat: -1.25888, lon: 36.77866)
 - Station 15 at (lat: -1.33024, lon: 36.88628)
 - Station 16 at (lat: -1.12069, lon: 37.00396)
 - Station 17 at (lat: -1.24114, lon: 36.68669)
 - Station 18 at (lat: -1.42951, lon: 37.64657)
 - Station 19 at (lat: -1.23007, lon: 36.87815)
 - Station 20 at (lat: -2.

In [13]:
print("\n Accident Assignments:")
for j in range(num_accidents):
    for i in range(num_stations):
        if y[(i, j)].varValue == 1:
            print(f" - Accident {j} handled by Station {i}")


 Accident Assignments:
 - Accident 0 handled by Station 16
 - Accident 1 handled by Station 76
 - Accident 2 handled by Station 76
 - Accident 3 handled by Station 85
 - Accident 4 handled by Station 16
 - Accident 5 handled by Station 76
 - Accident 6 handled by Station 16
 - Accident 7 handled by Station 92
 - Accident 8 handled by Station 77
 - Accident 9 handled by Station 65
 - Accident 10 handled by Station 25
 - Accident 11 handled by Station 89
 - Accident 12 handled by Station 8
 - Accident 13 handled by Station 90
 - Accident 14 handled by Station 62
 - Accident 15 handled by Station 22
 - Accident 16 handled by Station 57
 - Accident 17 handled by Station 70
 - Accident 18 handled by Station 57
 - Accident 19 handled by Station 70
 - Accident 20 handled by Station 57
 - Accident 21 handled by Station 73
 - Accident 22 handled by Station 63
 - Accident 23 handled by Station 63
 - Accident 24 handled by Station 58
 - Accident 25 handled by Station 8
 - Accident 26 handled by 

In [14]:
import folium
from folium.plugins import MarkerCluster

# Initialize map centered around average accident location
avg_lat = accident_df['latitude'].mean()
avg_lon = accident_df['longitude'].mean()
m = folium.Map(location=[avg_lat, avg_lon], zoom_start=12)

# Add Accident Points
accident_cluster = MarkerCluster(name="Accidents").add_to(m)
for idx, row in accident_df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color='red',
        fill=True,
        fill_opacity=0.6
    ).add_to(accident_cluster)

# Add Ambulance Stations
for i, (lat, lon) in enumerate(station_coords):
    if x[i].varValue == 1:
        folium.Marker(
            location=[lat, lon],
            icon=folium.Icon(color='blue', icon='plus-sign', prefix='glyphicon'),
            popup=f"Ambulance Station {i}"
        ).add_to(m)

# Optional: Draw lines showing assignment=
for j in range(num_accidents):
    for i in range(len(station_coords)):
        if y[(i, j)].varValue == 1:
            acc_lat, acc_lon = accident_coords[j]
            stat_lat, stat_lon = station_coords[i]
            folium.PolyLine(
                locations=[[stat_lat, stat_lon], [acc_lat, acc_lon]],
                color='gray',
                weight=1,
                opacity=0.5
            ).add_to(m)

# Add Layer Control
folium.LayerControl().add_to(m)

# Show map
m.save("ambulance_optimization_map.html")
print(" Map saved as 'ambulance_optimization_map.html'")


 Map saved as 'ambulance_optimization_map.html'
